In [ ]:
def get_mframeParameters(filepath):
    try:
        file=h5py.File(filepath,'r')
        #print("file = %s \n" % os.path.basename(filepath))
    except IOError:
        sys.stderr.write("File does not exist:\n  %s\n" % filepath)
        sys.exit(-1)
    
    phase = file['phase'][0]
    X = np.array(file.get('X'))
    Y = np.array(file.get('Y'))
    ex = np.array(file.get('ex'))
    n0 = np.array(file.get('n0'))
    n1 = np.array(file.get('n1'))
    n2 = np.array(file.get('n2'))
    ydim = Y.shape
    onaxisY2 = int(ydim[0]/2)
    ex_onaxis = ex[onaxisY2,:]
    distance = phase/kp     # [m]
    #print(lamdap, phase)
    return n0, n1, n2, ex, ex_onaxis, X, Y, distance

In [ ]:
from pathlib import Path

acc_Wakefield_scan_0cm = []
acc_Wakefield_scan_5cm = []
acc_Wakefield_scan_10cm = []
acc_Wakefield_scan_15cm = []
acc_Wakefield_scan_20cm = []
acc_Wakefield_scan_25cm = []

dec_Wakefield_scan_0cm = []
dec_Wakefield_scan_5cm = []
dec_Wakefield_scan_10cm = []
dec_Wakefield_scan_15cm = []
dec_Wakefield_scan_20cm = []
dec_Wakefield_scan_25cm = []

#minWakefield_scan_30cm = []

for h5files_dir in list_of_h5files_paths:
    
    # Determine the density
    # #####################
    density_str = h5files_dir.split('density-')[1].split('/')[0]
    density = float(density_str)

    print('density = ', "{:.2e}".format(density), end="\r")  

    density*=1e6 #/m3
    wp = np.sqrt(density*e**2/epsilon_0/m_e)
    kp=wp/c
    lamdap = 2*pi/kp
    E_WB = c*m_e*wp/e # wavebreaking field
    E_WB/=1e9 #GeV
    
    # Select files to analyse
    # #######################    
    h5files_dir = Path(h5files_dir) # Specify the folder path

    # Use Path.glob() to list files matching the pattern
    v2d_mframe_files = list(h5files_dir.glob('v2d_mframe_*.h5'))

    # Sort the filenames based on the numeric part extracted from each filename
    sorted_v2d_mframe_files = sorted(v2d_mframe_files)
    number_of_output_spots = len(sorted_v2d_mframe_files)


    # Iterate over sorted_v2d_mframe_file
    for i, mframe_file in enumerate(sorted_v2d_mframe_files):
        n0, n1, n2, ex, ex_onaxis, X, Y, distance = get_mframeParameters(mframe_file)

        abs_acc_wakefield = abs( min(ex_onaxis) ) * E_WB # in GV/m as E_WB is in GV/m
        dec_wakefield = max(ex_onaxis) * E_WB # in GV/m as E_WB is in GV/m
        if i == 0:
            acc_Wakefield_scan_0cm.append(abs_acc_wakefield)
            dec_Wakefield_scan_0cm.append(dec_wakefield)
        elif i == 1:
            acc_Wakefield_scan_5cm.append(abs_acc_wakefield)
            dec_Wakefield_scan_5cm.append(dec_wakefield)
        elif i == 2:
            acc_Wakefield_scan_10cm.append(abs_acc_wakefield)
            dec_Wakefield_scan_10cm.append(dec_wakefield)
        elif i == 3:
            acc_Wakefield_scan_15cm.append(abs_acc_wakefield)
            dec_Wakefield_scan_15cm.append(dec_wakefield)
        elif i == 4:
            acc_Wakefield_scan_20cm.append(abs_acc_wakefield)
            dec_Wakefield_scan_20cm.append(dec_wakefield)
        elif i == 5:
            acc_Wakefield_scan_25cm.append(abs_acc_wakefield)
            dec_Wakefield_scan_25cm.append(dec_wakefield)
#        elif i == 6:
#            minWakefield_scan_30cm.append(abs_min_wakefield)
            
# Plot maximum wakefield
# ######################
fig, ax = plt.subplots(figsize=figsize)
#fig, axs = plt.subplots(2, 1, figsize=(6, 8))

###################################
#ax = axs[0]
# Customize tick parameters
ax.tick_params(which='both', width=1.2)
ax.tick_params(which='major', length=5)
ax.tick_params(which='minor', length=2)

# Scatter plot with colors and sizes
#ax.scatter(density_array, minWakefield_scan_0cm, c='k', s=14, alpha=1, marker='x', label = '0 cm')
#plt.plot(density_array, minWakefield_scan_0cm, color='orange', linestyle='-', linewidth=2)

#ax.scatter(density_array, minWakefield_scan_5cm, c='r', s=14, alpha=1, marker='s', label = '5 cm')
#plt.plot(density_array, minWakefield_scan_5cm, color='orange', linestyle='-', linewidth=2)

#ax.scatter(density_array, minWakefield_scan_10cm, c='b', s=14, alpha=1, marker='^')
ax.plot(density_array, acc_Wakefield_scan_10cm, color=colors[0], linestyle='-', linewidth=2, label = '10 cm')
ax.axhline(y=2.5, color=colors[0], linewidth=1, ls='--', label = ' ')  # Add a horizontal line at y=0

#ax.scatter(density_array, minWakefield_scan_15cm, c='g', s=14, alpha=1, marker='o')
ax.plot(density_array, acc_Wakefield_scan_15cm, color=colors[1], linestyle='-', linewidth=2, label = '15 cm')
ax.axhline(y=1.7, color=colors[1], linewidth=1, ls='--', label = ' ')  # Add a horizontal line at y=0

#ax.scatter(density_array, minWakefield_scan_20cm, c='c', s=14, alpha=1, marker='^'
ax.plot(density_array, acc_Wakefield_scan_20cm, color=colors[2], linestyle='-', linewidth=2, label = '20 cm')
ax.axhline(y=1.25, color=colors[2], linewidth=1, ls='--', label = ' ')  # Add a horizontal line at y=0

#ax.scatter(density_array, minWakefield_scan_25cm, c='k', s=14, alpha=1, marker='d')
ax.plot(density_array, acc_Wakefield_scan_25cm, color=colors[3], linestyle='-', linewidth=2, label = '25 cm')
ax.axhline(y=1, color=colors[3], linewidth=1, ls='--', label = ' ')  # Add a horizontal line at y=0

#ax.scatter(density_array, minWakefield_scan_20cm, c='c', s=14, alpha=1, marker='^'
#plt.plot(density_array, minWakefield_scan_30cm, color=colors[4], linestyle='-', linewidth=2, label = '30 cm')
#plt.axhline(y=1.25, color=colors[4], linewidth=1, ls='--', label = ' ')  # Add a horizontal line at y=0

ax.set_ylim(0, 3.5)
ax.set_xlim(5e14, 1e17)
ax.tick_params(axis='both', which='major', labelsize=fontsize14)  # Change 14 to your desired font size

#ax.set_xlim(density_array[0], density_array[-1])
#ax.tick_params(axis='both', which='major', labelsize=fontsize)  # Change 14 to your desired font size

ax.legend(ncol=2, loc='upper left', fontsize=fontsize)

#ax.set_xlabel(r'$\mathrm{Plasma \, density,} \, n_0 \, [\mathrm{cm}^{-3}]$', fontsize=fontsize)
ax.set_ylabel(r'$\mathrm{Max.} \, E_\mathrm{acc} \, [Gv/m]$', fontsize=fontsize14)
ax.set_xscale('log')
#plt.yscale('log')
#ax.grid(True, which="both", axis='x')

# Add a shaded grey region between x1 and x2
ax.axvspan(best_density_range[0], best_density_range[1], color='grey', alpha=0.3, label='Shaded Region')

# Save the plot as both PDF and PNG
#plt.savefig('maxWakefield' + '.pdf', dpi=100, bbox_inches='tight')
#plt.savefig('maxWakefield' + '.png', dpi=100, bbox_inches='tight')

# Show the plot (optional, depending on the environment)
plt.show()
   